In [1]:
import os
import time as t

import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

In [2]:
# 사용할 변수명

cc = [
    "subject", "age", "group", "time",
    "R_HIP_Flex_ANG", "L_HIP_Flex_ANG",
    "R_KNEE_Flex_ANG", "L_KNEE_Flex_ANG",
    "R_ANK_Flex_ANG", "L_ANK_Flex_ANG",
    "R_Pelvis_Lat_Tilt", "L_Pelvis_Lat_Tilt",
    "R_HIP_Rot_ANG", "L_HIP_Rot_ANG",
    "R_Foot_Orientation", "L_Foot_Orientation"
]

In [3]:
# raw data가 들어있는 폴더명
# .(콤마)는 현위치를 의미. 현 위치의 하위 폴더 ./폴더명

path_false = "./data/20190709/"
path_true = "./data/20190709/"

In [4]:
# xlsx 파일에서 데이터와 age를 읽어오는 함수

def read_files(path, file):
    data = pd.read_excel(
        os.path.join(path, file),
        sheet_name="Average",
        header=1,
        skiprows=27,)
    try:
        age = pd.read_excel(
            os.path.join(path, file),
            skipfooter=17,
            header=1,
        )["Unnamed: 16"][6]
    except Exception as e:
        print(file)
        age = np.nan
    return data, age

In [5]:
# 파일 저장용 함수

def save_data(result, group: str = "all"):
    now = t.localtime()
    folder_name = "result"
    try:
        if not(os.path.isdir(folder_name)):
            os.makedirs(os.path.join(folder_name))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

    s = "%04d%02d%02d_%02d%02d" % (now.tm_year, now.tm_mon,
                                   now.tm_mday, now.tm_hour, now.tm_min)

    # 파일 저장
    result.to_csv("./{}/gait_{}_{}.csv".format(folder_name, s, group),
                  encoding='cp949', index=False)

In [6]:
# 파일 읽는 순서 상관 있는지? 

# key=lambda x: int(x.split(".")[0]))
files_false = sorted(os.listdir(path_false),)
files_false = [f for f in files_false if ".xl" in f] # 엑셀 파일만 가능하도록
files_false

['001.xls', '002.xls', '003.xls']

In [14]:
%%time

# 일반적인 table로 생성 후, 가로로 붙이는 작업을 한다.

time = np.arange(1, 101)
subject = 0
result = pd.DataFrame()

for file in files_false:
    print(file)
    data, age = read_files(path_false, file)
    group = 0
    data["subject"] = subject
    data["age"] = age
    data["group"] = group
    data["time"] = time
    data = data[cc]

    if result.empty:
        cols = cc
        c = cc[3:]

        # make colums
        col_result = []
        for n in range(1, 101):
            col_result.extend(list(map(lambda x: x+"_{:03}".format(n), c)))
        kkk = cols[:3]+col_result
        result = pd.DataFrame(columns=kkk)

    # make data to longitudinal
    longi_data = []
    for aa in data[data.columns[3:]].values.tolist():
        longi_data.extend(aa)
        
    temp = pd.DataFrame([[subject,  age, group, ] + longi_data], columns=kkk)
    result = result.append(temp, ignore_index=True)

    subject += 1

001.xls
    subject        age  group  time  R_HIP_Flex_ANG  L_HIP_Flex_ANG  \
0         0  18.315068      0     1       35.994166       38.230834   
1         0  18.315068      0     2       35.936834       38.220354   
2         0  18.315068      0     3       35.766380       38.094608   
3         0  18.315068      0     4       35.505561       37.809546   
4         0  18.315068      0     5       35.192071       37.468276   
5         0  18.315068      0     6       34.842748       37.189536   
6         0  18.315068      0     7       34.444232       36.949359   
7         0  18.315068      0     8       33.947361       36.634441   
8         0  18.315068      0     9       33.331378       36.218969   
9         0  18.315068      0    10       32.622682       35.693211   
10        0  18.315068      0    11       31.818368       35.014945   
11        0  18.315068      0    12       30.921481       34.201982   
12        0  18.315068      0    13       29.920672       33.266479  

    subject        age  group  time  R_HIP_Flex_ANG  L_HIP_Flex_ANG  \
0         1  18.863014      0     1       32.453832       29.999500   
1         1  18.863014      0     2       32.648283       30.423406   
2         1  18.863014      0     3       32.824886       30.804704   
3         1  18.863014      0     4       32.972581       31.084222   
4         1  18.863014      0     5       33.066740       31.260094   
5         1  18.863014      0     6       33.069616       31.382158   
6         1  18.863014      0     7       32.627638       31.180333   
7         1  18.863014      0     8       32.223057       30.912333   
8         1  18.863014      0     9       31.717470       30.512334   
9         1  18.863014      0    10       30.984712       29.973636   
10        1  18.863014      0    11       30.152313       29.350753   
11        1  18.863014      0    12       29.532777       28.762666   
12        1  18.863014      0    13       28.886515       28.070823   
13    

    subject        age  group  time  R_HIP_Flex_ANG  L_HIP_Flex_ANG  \
0         2  14.073973      0     1       37.593667       36.241666   
1         2  14.073973      0     2       37.586651       35.884179   
2         2  14.073973      0     3       37.432048       35.495360   
3         2  14.073973      0     4       37.079475       35.047555   
4         2  14.073973      0     5       36.589102       34.575075   
5         2  14.073973      0     6       36.093872       34.169243   
6         2  14.073973      0     7       35.731080       33.888020   
7         2  14.073973      0     8       35.543655       33.736113   
8         2  14.073973      0     9       35.399068       33.566713   
9         2  14.073973      0    10       35.113697       33.205031   
10        2  14.073973      0    11       34.534323       32.498259   
11        2  14.073973      0    12       33.596999       31.407555   
12        2  14.073973      0    13       32.360001       30.043636   
13    

In [8]:
# save false
save_data(result=result, group="false")

In [10]:
result

,subject,age,group,time_001,R_HIP_Flex_ANG_001,L_HIP_Flex_ANG_001,R_KNEE_Flex_ANG_001,L_KNEE_Flex_ANG_001,R_ANK_Flex_ANG_001,L_ANK_Flex_ANG_001,...,R_KNEE_Flex_ANG_100,L_KNEE_Flex_ANG_100,R_ANK_Flex_ANG_100,L_ANK_Flex_ANG_100,R_Pelvis_Lat_Tilt_100,L_Pelvis_Lat_Tilt_100,R_HIP_Rot_ANG_100,L_HIP_Rot_ANG_100,R_Foot_Orientation_100,L_Foot_Orientation_100
0,0,18.315068,0,1.0,35.994166,38.230834,10.060,9.5480,2.963667,4.415333,...,8.416667,8.387833,3.682833,4.490000,0.0875,0.578500,-1.697333,-0.720167,-10.691167,-8.707333
1,1,18.863014,0,1.0,32.453832,29.999500,11.017,3.0565,1.543333,0.720500,...,6.215667,-1.092333,2.091167,1.300167,-0.1130,-1.822333,17.000167,15.388500,-10.560333,-13.918834


In [11]:
# 파일 읽는 순서 상관 있는지?

# key=lambda x: int(x.split(".")[0]))
files_true = sorted(os.listdir(path_true),)
files_true = [f for f in files_true if ".xl" in f] # 엑셀 파일만 가능하도록
files_true

['001.xls', '002.xls']

In [12]:
%%time

time = np.arange(1, 101)

result_true = pd.DataFrame()

for file in files_true:
    print(file)
    data, age = read_files(path_true, file)
    group = 1
    data["subject"] = subject
    data["age"] = age
    data["group"] = group
    data["time"] = time
    data = data[cc]
    
    if result_true.empty:
        cols = cc
        c = cc[3:]

        # make colums
        col_result = []
        for n in range(1, 101):
            col_result.extend(list(map(lambda x: x+"_{:03}".format(n), c)))
        kkk = cols[:3]+col_result
        result_true = pd.DataFrame(columns=kkk)

    # make data to longitudinal
    longi_data = []
    for aa in data[data.columns[3:]].values.tolist():
        longi_data.extend(aa)

    temp = pd.DataFrame([[subject,  age, group, ] + longi_data], columns=kkk)
    result_true = result_true.append(temp, ignore_index=True)

    subject += 1

001.xls
002.xls
Wall time: 6.21 s


In [13]:
# save True
save_data(result=result_true, group="true")

In [14]:
# merge
# time 변수 정수화
r = result.append(result_true, ignore_index=True, sort=False)
r[["time_{:03}".format(x) for x in range(1, 101)]] = r[[
    "time_{:03}".format(x) for x in range(1, 101)]].applymap(int)

In [15]:
# save All
save_data(result=r, group="all")

In [16]:
result_true[:3]

,subject,age,group,time_001,R_HIP_Flex_ANG_001,L_HIP_Flex_ANG_001,R_KNEE_Flex_ANG_001,L_KNEE_Flex_ANG_001,R_ANK_Flex_ANG_001,L_ANK_Flex_ANG_001,...,R_KNEE_Flex_ANG_100,L_KNEE_Flex_ANG_100,R_ANK_Flex_ANG_100,L_ANK_Flex_ANG_100,R_Pelvis_Lat_Tilt_100,L_Pelvis_Lat_Tilt_100,R_HIP_Rot_ANG_100,L_HIP_Rot_ANG_100,R_Foot_Orientation_100,L_Foot_Orientation_100
0,2,18.315068,1,1.0,35.994166,38.230834,10.060,9.5480,2.963667,4.415333,...,8.416667,8.387833,3.682833,4.490000,0.0875,0.578500,-1.697333,-0.720167,-10.691167,-8.707333
1,3,18.863014,1,1.0,32.453832,29.999500,11.017,3.0565,1.543333,0.720500,...,6.215667,-1.092333,2.091167,1.300167,-0.1130,-1.822333,17.000167,15.388500,-10.560333,-13.918834
